In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt

In [2]:
# Retreiving active rosters

# Instantiating necessary items
tables = pd.read_html('https://www.lineups.com/nba/depth-charts')
teams = ['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets',
        'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets',
        'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
        'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat',
        'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans',
        'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers',
        'Phoenix Suns', 'Portland Trailblazers', 'Sacramento Kings', 'San Antonio Spurs',
        'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
def name_adjustment(x):
    try:
        names = x.split(' ')
        if len(names) == 4:
            name = names[0] + ' ' + names[1]
        if len(names) == 6:
            name = names[0] + ' ' + names[1] + ' ' + names[2]
    except:
        name = x
    return name

# Getting active rosters into dictionary of lists for each team
team_dict = {}
for team, table in zip(teams,tables):
    table.columns = table.columns.droplevel(0)
    for i in [1,2,3]:
        table[str(i)] = table[str(i)].apply(name_adjustment)
    table = table[['1', '2', '3']]
    players = list()
    for i in range(table.shape[0]): 
        for j in range(table.shape[1]):
            player = table.iloc[i, j]
            players = players + [player]
    team_dict[team] = players

In [10]:
# Retreiving games played table

# Map to change team names
games_played_map = {
    'Milwaukee' : 'Milwaukee Bucks',
    'Boston' : 'Boston Celtics',
    'Memphis' : 'Memphis Grizzlies',
    'LA Clippers' : 'Los Angeles Clippers',
    'Sacramento' : 'Sacramento Kings',
    'Indiana' : 'Indiana Pacers',
    'Golden State' : 'Golden State Warriors',
    'LA Lakers' : 'Los Angeles Lakers',
    'San Antonio' : 'San Antonio Spurs',
    'Utah' : 'Utah Jazz',
    'Houston' : 'Houston Rockets',
    'Dallas' : 'Dallas Mavericks',
    'Charlotte' : 'Charlotte Hornets',
    'Cleveland' : 'Cleveland Cavaliers',
    'Orlando' : 'Orlando Magic',
    'Washington' : 'Washington Wizards',
    'New York' : 'New York Knicks',
    'Brooklyn' : 'Brooklyn Nets',
    'Philadelphia' : 'Philadelphia 76ers',
    'Minnesota' : 'Minnesota Timberwolves',
    'Miami' : 'Miami Heat',
    'New Orleans' : 'New Orleans Pelicans',
    'Detroit' : 'Detroit Pistons',
    'Okla City' : 'Oklahoma City Thunder',
    'Portland' : 'Portland Trailblazers',
    'Chicago' : 'Chicago Bulls',
    'Denver' : 'Denver Nuggets',
    'Phoenix' : 'Phoenix Suns',
    'Atlanta' : 'Atlanta Hawks',
    'Toronto' : 'Toronto Raptors'
}

# Reading in table and adjusting columns
tables = pd.read_html('https://www.teamrankings.com/nba/stat/games-played')
games_played_table = tables[0]
games_played_table = games_played_table[['Team', str(current_year - 1)]]
games_played_table.columns = ['Team', 'Games_Played']
games_played_table['Team'] = games_played_table.Team.apply(lambda x: games_played_map[x])

frac_season = np.mean(games_played_table.Games_Played)

/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
current_year = 2022

# Retrieving current year VORPs
tables = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{str(current_year)}_advanced.html')
table = tables[0]
table = table[['Player', 'Tm', 'G', 'VORP']]
table.columns = ['Player', 'Team', 'Games', 'VORP']
table = table[table.Team != 'Tm']
table = table[table.Team != 'TOT']
table['VORP'] = table.VORP.apply(pd.to_numeric)
player_vorp = table.groupby(['Player', 'Games'])['VORP'].sum()
player_vorp = pd.DataFrame(player_vorp)
player_vorp.reset_index(drop = False, inplace = True)
player_vorp.columns = ['Player', 'Games', 'VORP']

,Player,Games,VORP
0,Aaron Gordon,44,0.6
1,Aaron Henry,6,-0.1
2,Aaron Holiday,36,0.0
3,Aaron Nesmith,33,-0.3
4,Aaron Wiggins,29,-0.1
...,...,...,...
613,Yves Pons,6,0.0
614,Zach LaVine,40,1.8
615,Zeke Nnaji,31,0.0
616,Ziaire Williams,31,-0.5


48.2
